<a href="https://colab.research.google.com/github/billvo2212/reproducing-research-paper-result/blob/main/RSICD_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Core stack (CUDA 12.1 wheels work well on Colab A100/L4/T4)
!pip -q install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --extra-index-url https://download.pytorch.org/whl/cu121
!pip -q install transformers==4.41.2 accelerate==0.31.0 peft==0.11.1 bitsandbytes==0.43.1 sentencepiece
!pip -q install decord==0.6.0 datasets==2.20.0 rouge-score==0.1.2 pycocoevalcap

# Java (if you want METEOR/SPICE original implementations)
!apt-get -yqq update && apt-get -yqq install openjdk-17-jre-headless

# Install additional evaluation dependencies
!pip install -q nltk

# Install iopath and other potentially missing packages
!pip install -q iopath omegaconf timm einops

# Install additional packages that RSGPT might need
!pip install -q webdataset braceexpand

!apt-get update -qq
!apt-get install -y default-jdk -qq

import os
os.environ['_JAVA_OPTIONS'] = '-Xmx8g'

# Login to Hugging Face if your base LLM requires it
from huggingface_hub import notebook_login
notebook_login()  # or set HF_TOKEN env var

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/data/RSGPT"                      # repo path
DATA_ROOT    = "/content/drive/MyDrive/data/rsgpt"   # place your datasets here
OUT_ROOT     = "/content/drive/MyDrive/outputs/rsgpt"

!mkdir -p {DATA_ROOT}/rsicap/images
!mkdir -p {DATA_ROOT}/rsicap
!mkdir -p {DATA_ROOT}/rsieval/images
!mkdir -p {DATA_ROOT}/rsieval
!mkdir -p {OUT_ROOT}

print(f"✓ Directories created:")
print(f"  Project: {PROJECT_ROOT}")
print(f"  Data: {DATA_ROOT}")
print(f"  Output: {OUT_ROOT}")

print("\n📁 Expected data structure:")
print(f"""
{DATA_ROOT}/
  ├── rsicap/
  │   ├── images/          # Training images
  │   └── annotations.json # Training annotations
  └── rsieval/
      ├── images/          # Evaluation images
      ├── captions.json    # Caption evaluation
      └── vqa.json         # VQA evaluation (if applicable)
""")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Directories created:
  Project: /content/drive/MyDrive/data/RSGPT
  Data: /content/drive/MyDrive/data/rsgpt
  Output: /content/drive/MyDrive/outputs/rsgpt

📁 Expected data structure:

/content/drive/MyDrive/data/rsgpt/
  ├── rsicap/
  │   ├── images/          # Training images
  │   └── annotations.json # Training annotations
  └── rsieval/
      ├── images/          # Evaluation images
      ├── captions.json    # Caption evaluation
      └── vqa.json         # VQA evaluation (if applicable)



In [ ]:
# @title
# Helper to print what the repo expects for model_type and datasets
import re
import pathlib
import sys

sys.path.insert(0, PROJECT_ROOT)

root = pathlib.Path(f"{PROJECT_ROOT}/rsgpt")

# Find valid model types
valid_models = []
for p in root.rglob("*.py"):
    try:
        t = p.read_text(encoding="utf-8", errors="ignore")
        if "PRETRAINED_MODEL_CONFIG_DICT" in t:
            m = re.search(r"PRETRAINED_MODEL_CONFIG_DICT\s*=\s*({.*?})", t, re.S)
            if m:
                keys = re.findall(r"['\"]([^'\"]+)['\"]\s*:", m.group(1))
                valid_models.extend(keys)
    except:
        pass

# Find valid dataset builders
valid_datasets = []
for p in root.rglob("*.py"):
    try:
        t = p.read_text(encoding="utf-8", errors="ignore")
        found = re.findall(r'register_builder\(\s*[\'"]([^\'"]+)[\'"]\s*\)', t)
        valid_datasets.extend(found)
    except:
        pass

print("📋 Available model types:")
for i, model in enumerate(sorted(set(valid_models)), 1):
    print(f"  {i}. {model}")

print("\n📋 Available dataset builders:")
for i, dataset in enumerate(sorted(set(valid_datasets))[:20], 1):
    print(f"  {i}. {dataset}")

print("\n💡 Choose your model_type from the list above")


📋 Available model types:
  1. vicuna13b
  2. vicuna7b

📋 Available dataset builders:
  1. cc_sbu_align
  2. rsicap_instruction
  3. rsicd
  4. rsicd_instruction
  5. rsvqahr_instruction
  6. rsvqalr_instruction
  7. sydney_instruction
  8. ucm_instruction

💡 Choose your model_type from the list above


# STEP 1: DOWNLOAD RSICD DATASET

In [ ]:
# @title Step 2.1: Download RSICD Dataset
import os

print("="*60)
print("Step 2.1: Download RSICD Dataset")
print("="*60)

# Create directory
rsicd_base = "/content/drive/MyDrive/data/RSGPT/dataset/rsicd"
os.makedirs(rsicd_base, exist_ok=True)

# Clone RSICD repository
%cd /content
!rm -rf RSICD_optimal  # Remove if exists
!git clone https://github.com/201528014227051/RSICD_optimal.git

# Check contents
print("\n✓ Downloaded RSICD repository")
print("\nContents:")
!ls -la /content/RSICD_optimal

Step 2.1: Download RSICD Dataset
/content
Cloning into 'RSICD_optimal'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 65 (delta 4), reused 0 (delta 0), pack-reused 56 (from 1)
Receiving objects: 100% (65/65), 1.37 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (35/35), done.
Error downloading object: RSICD_images.zip (0d3ebda): Smudge error: Error downloading RSICD_images.zip (0d3ebda15b8b1e5c2754270d38462251e8c61489b54e463e499c978da4b91d10): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/RSICD_optimal/.git/lfs/logs/20251230T161536.842484838.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: RSICD_images.zip: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --so

# STEP 2: PREPARE RSICD DATA

In [ ]:
# @title Check RSICD Annotation Format
import json
import os

src_path = "/content/RSICD_optimal"

# Find JSON files
print("JSON files in RSICD_optimal:")
for f in os.listdir(src_path):
    if f.endswith('.json'):
        print(f"  - {f}")

# Load and inspect
ann_file = None
for f in ["dataset_rsicd.json", "rsicd.json", "annotations.json"]:
    if os.path.exists(f"{src_path}/{f}"):
        ann_file = f"{src_path}/{f}"
        break

if ann_file:
    with open(ann_file) as f:
        data = json.load(f)

    print(f"\nLoaded: {ann_file}")
    print(f"Type: {type(data)}")

    if isinstance(data, dict):
        print(f"Keys: {data.keys()}")

        # Check images structure
        if "images" in data:
            print(f"\nImages count: {len(data['images'])}")
            print(f"Sample image entry:")
            print(json.dumps(data['images'][0], indent=2))

        # Check annotations structure
        if "annotations" in data:
            print(f"\nAnnotations count: {len(data['annotations'])}")
            print(f"Sample annotation entry:")
            print(json.dumps(data['annotations'][0], indent=2))

    elif isinstance(data, list):
        print(f"List length: {len(data)}")
        print(f"Sample entry:")
        print(json.dumps(data[0], indent=2))
else:
    print("No annotation file found!")
    print(f"\nContents of {src_path}:")
    !ls -la {src_path}

JSON files in RSICD_optimal:
  - dataset_rsicd.json

Loaded: /content/RSICD_optimal/dataset_rsicd.json
Type: <class 'dict'>
Keys: dict_keys(['images', 'dataset'])

Images count: 10921
Sample image entry:
{
  "filename": "airport_1.jpg",
  "imgid": 0,
  "sentences": [
    {
      "tokens": [
        "many",
        "planes",
        "are",
        "parked",
        "next",
        "to",
        "a",
        "long",
        "building",
        "in",
        "an",
        "airport"
      ],
      "raw": "many planes are parked next to a long building in an airport .",
      "imgid": 0,
      "sentid": 0
    },
    {
      "tokens": [
        "many",
        "planes",
        "are",
        "parked",
        "next",
        "to",
        "a",
        "long",
        "building",
        "in",
        "an",
        "airport"
      ],
      "raw": "many planes are parked next to a long building in an airport .",
      "imgid": 0,
      "sentid": 1
    },
    {
      "tokens": [
        "many"

In [ ]:
# @title Step 2.2: Prepare RSICD Data (Fixed for RSICD Format)
import json
import os
import shutil
import random
from collections import defaultdict

print("="*60)
print("Step 2.2: Prepare RSICD Data")
print("="*60)

src_path = "/content/RSICD_optimal"
dst_path = "/content/drive/MyDrive/data/RSGPT/dataset/rsicd"

# os.makedirs(f"{dst_path}/image", exist_ok=True)

# ============================================
# [1] Copy images
# ============================================
# print("\n[1] Copying images...")

# img_src = None
# for folder in ["RSICD_images", "images", "RSICD"]:
#     check_path = f"{dst_path}/{folder}"
#     if os.path.exists(check_path):
#         files = os.listdir(check_path)
#         img_files = [f for f in files if f.lower().endswith(('.jpg', '.png', '.jpeg', '.tif'))]
#         if len(img_files) > 100:
#             img_src = check_path
#             break

# if img_src:
#     img_files = [f for f in os.listdir(img_src) if f.lower().endswith(('.jpg', '.png', '.jpeg', '.tif'))]
#     print(f"  Found {len(img_files)} images in {img_src}")
#     for i, f in enumerate(img_files):
#         shutil.copy(f"{img_src}/{f}", f"{dst_path}/image/{f}")
#         if (i+1) % 2000 == 0:
#             print(f"  Copied {i+1}/{len(img_files)}")
#     print(f"  ✓ Copied {len(img_files)} images")
# else:
#     print("  ⚠️ Images folder not found, searching...")
#     !find {src_path} -name "*.jpg" | head -5

# ============================================
# [2] Load annotations
# ============================================
print("\n[2] Loading annotations...")

with open(f"{src_path}/dataset_rsicd.json") as f:
    data = json.load(f)

images_data = data["images"]
print(f"  ✓ Loaded {len(images_data)} images")

# ============================================
# [3] Parse RSICD format
# ============================================
print("\n[3] Parsing RSICD format...")

# RSICD format:
# {
#   "filename": "airport_1.jpg",
#   "imgid": 0,
#   "sentences": [{"raw": "caption text", ...}, ...],
#   "split": "train" / "val" / "test"
# }

prompts = [
    "Describe this image in detail.",
    "Please provide a detailed description of the picture.",
    "Could you describe the contents of this image for me?",
    "Write a short description for the image.",
    "Take a look at this image and describe what you notice.",
]

# Use RSICD's original splits
splits = {"train": [], "val": [], "test": []}

for img in images_data:
    img_id = img["imgid"]
    filename = img["filename"]
    split = img["split"]

    # Extract captions from sentences
    captions = [sent["raw"] for sent in img["sentences"]]

    annotation = {
        "image_id": img_id,
        "filename": filename,
        "text_input": random.choice(prompts),
        "text_output": captions  # List of all captions
    }

    if split in splits:
        splits[split].append(annotation)
    else:
        # Handle unexpected split names
        splits["train"].append(annotation)

print(f"  Train: {len(splits['train'])} images")
print(f"  Val: {len(splits['val'])} images")
print(f"  Test: {len(splits['test'])} images")

# Show sample
sample = splits["train"][0]
print(f"\n  Sample annotation:")
print(f"    image_id: {sample['image_id']}")
print(f"    filename: {sample['filename']}")
print(f"    text_input: {sample['text_input']}")
print(f"    text_output: {len(sample['text_output'])} captions")
print(f"    caption[0]: {sample['text_output'][0]}")

# ============================================
# [4] Save annotation files
# ============================================
print("\n[4] Saving annotation files...")

for split_name, annotations in splits.items():
    out_file = f"{dst_path}/rsicd_cap_processed_instruction_{split_name}.json"
    with open(out_file, "w") as f:
        json.dump({"annotations": annotations}, f, indent=2)
    print(f"  ✓ {split_name}: {len(annotations)} images -> {out_file}")

# ============================================
# [5] Create COCO GT for evaluation
# ============================================
print("\n[5] Creating COCO GT files...")

for split_name in ["val", "test"]:
    annotations_list = splits[split_name]

    images = []
    coco_annotations = []
    ann_id = 1

    for ann in annotations_list:
        img_id = ann["image_id"]
        filename = ann["filename"]
        captions = ann["text_output"]

        images.append({"id": img_id, "file_name": filename})

        for cap in captions:
            coco_annotations.append({
                "id": ann_id,
                "image_id": img_id,
                "caption": cap
            })
            ann_id += 1

    coco_gt = {
        "info": {"description": f"RSICD {split_name}", "version": "1.0", "year": 2024},
        "licenses": [],
        "type": "captions",
        "images": images,
        "annotations": coco_annotations
    }

    gt_file = f"{dst_path}/rsicd_{split_name}_gt.json"
    with open(gt_file, "w") as f:
        json.dump(coco_gt, f)
    print(f"  ✓ {split_name} GT: {len(images)} images, {len(coco_annotations)} captions")

# ============================================
# [6] Verify final structure
# ============================================
print("\n[6] Verifying final structure...")

print(f"\n  Contents of {dst_path}:")
for item in sorted(os.listdir(dst_path)):
    item_path = f"{dst_path}/{item}"
    if os.path.isdir(item_path):
        count = len(os.listdir(item_path))
        print(f"    📁 {item}/ ({count} files)")
    else:
        size = os.path.getsize(item_path) / 1024
        print(f"    📄 {item} ({size:.1f} KB)")

# Verify image exists
sample_filename = splits["train"][0]["filename"]
img_path = f"{dst_path}/image/{sample_filename}"
print(f"\n  Sample image exists: {os.path.exists(img_path)} ({img_path})")


print("\n" + "="*60)
print("✅ Step 2.2 Complete!")
print("="*60)
print(f"""
Final structure:
{dst_path}/
├── image/                                         ← Images
├── rsicd_cap_processed_instruction_train.json     ← Train ({len(splits['train'])} images)
├── rsicd_cap_processed_instruction_val.json       ← Val ({len(splits['val'])} images)
├── rsicd_cap_processed_instruction_test.json      ← Test ({len(splits['test'])} images)
├── rsicd_val_gt.json                              ← COCO GT for val
└── rsicd_test_gt.json                             ← COCO GT for test
""")

Step 2.2: Prepare RSICD Data

[2] Loading annotations...
  ✓ Loaded 10921 images

[3] Parsing RSICD format...
  Train: 8734 images
  Val: 1094 images
  Test: 1093 images

  Sample annotation:
    image_id: 0
    filename: airport_1.jpg
    text_input: Take a look at this image and describe what you notice.
    text_output: 5 captions
    caption[0]: many planes are parked next to a long building in an airport .

[4] Saving annotation files...
  ✓ train: 8734 images -> /content/drive/MyDrive/data/RSGPT/dataset/rsicd/rsicd_cap_processed_instruction_train.json
  ✓ val: 1094 images -> /content/drive/MyDrive/data/RSGPT/dataset/rsicd/rsicd_cap_processed_instruction_val.json
  ✓ test: 1093 images -> /content/drive/MyDrive/data/RSGPT/dataset/rsicd/rsicd_cap_processed_instruction_test.json

[5] Creating COCO GT files...
  ✓ val GT: 1094 images, 5470 captions
  ✓ test GT: 1093 images, 5465 captions

[6] Verifying final structure...

  Contents of /content/drive/MyDrive/data/RSGPT/dataset/rsicd:


# STEP 3: CREATE RSICD TRAINING CONFIG

In [ ]:
# @title Step 2.4: Create RSICD Training Config
import os

print("="*60)
print("Step 2.4: Create RSICD Training Config")
print("="*60)

# ============================================
# SET YOUR BEST RSICAP CHECKPOINT HERE
# ============================================
# Update this path to your best RSICap checkpoint
# BEST_RSICAP_CHECKPOINT = "/content/drive/MyDrive/outputs/rsgpt/rsicap_extended/YYYYMMDD/checkpoint_XX.pth"

# Or use the original best checkpoint
BEST_RSICAP_CHECKPOINT = "/content/drive/MyDrive/outputs/rsgpt/rsicap_split_checkpoint/20251229133/checkpoint_8.pth"

print(f"Using RSICap checkpoint: {BEST_RSICAP_CHECKPOINT}")

# ============================================
# Create RSICD Training Config
# ============================================
rsicd_train_config = f'''model:
  arch: rsgpt
  model_type: vicuna13b
  freeze_vit: True
  freeze_qformer: False
  max_txt_len: 160
  llm_model: lmsys/vicuna-13b-v1.5
  end_sym: "###"
  prompt_path: "prompts/alignment.txt"

datasets:
  rsicd_instruction:
    vis_processor:
      train:
        name: "rs_image_train"
        image_size: 224
    text_processor:
      train:
        name: "blip_caption"

run:
  task: image_text_pretrain
  lr_sched: "linear_warmup_cosine_lr"
  init_lr: 5e-6                    # Lower LR for fine-tuning
  min_lr: 1e-7
  warmup_lr: 1e-8
  weight_decay: 0.05

  max_epoch: 15                    # Fine-tuning epochs
  iters_per_epoch: 200             # Adjust based on RSICD train size
  batch_size_train: 8
  batch_size_eval: 4
  num_workers: 8
  warmup_steps: 150

  seed: 42
  output_dir: "/content/drive/MyDrive/outputs/rsgpt/rsicd_finetuned"

  amp: True
  resume_ckpt_path: "{BEST_RSICAP_CHECKPOINT}"

  evaluate: False
  train_splits: ["train"]

  device: "cuda"
  world_size: 1
  dist_url: "env://"
  distributed: True
'''

# Save config
config_path = "/content/drive/MyDrive/data/RSGPT/train_configs/rsicd_train.yaml"
with open(config_path, "w") as f:
    f.write(rsicd_train_config)

print(f"✓ Created RSICD training config: {config_path}")

# ============================================
# Verify checkpoint exists
# ============================================
if os.path.exists(BEST_RSICAP_CHECKPOINT):
    print(f"✓ Checkpoint exists: {BEST_RSICAP_CHECKPOINT}")
else:
    print(f"⚠️ WARNING: Checkpoint not found: {BEST_RSICAP_CHECKPOINT}")
    print("  Please update BEST_RSICAP_CHECKPOINT with the correct path")

print("\n" + "="*60)
print("Step 2.4 Complete!")
print("="*60)

Step 2.4: Create RSICD Training Config
Using RSICap checkpoint: /content/drive/MyDrive/outputs/rsgpt/rsicap_split_checkpoint/20251229133/checkpoint_8.pth
✓ Created RSICD training config: /content/drive/MyDrive/data/RSGPT/train_configs/rsicd_train.yaml
✓ Checkpoint exists: /content/drive/MyDrive/outputs/rsgpt/rsicap_split_checkpoint/20251229133/checkpoint_8.pth

Step 2.4 Complete!


# STEP 4: CREATE RSICD EVALUATION CONFIG

In [ ]:
# @title Step 2.5: Create RSICD Evaluation Config
import os

print("="*60)
print("Step 2.5: Create RSICD Evaluation Config")
print("="*60)

# ============================================
# Create RSICD Evaluation Config
# ============================================
rsicd_eval_config = '''model:
  arch: rsgpt
  model_type: vicuna13b
  max_txt_len: 160
  llm_model: lmsys/vicuna-13b-v1.5
  end_sym: "###"
  pretrained:

datasets:
  rsicd_instruction:
    vis_processor:
      train:
        name: "rs_image_train"
        image_size: 224
    text_processor:
      train:
        name: "blip_caption"

run:
  task: image_text_pretrain
'''

# Save config
eval_config_path = "/content/drive/MyDrive/data/RSGPT/eval_configs/rsicd_eval.yaml"
with open(eval_config_path, "w") as f:
    f.write(rsicd_eval_config)

print(f"✓ Created RSICD eval config: {eval_config_path}")

print("\n" + "="*60)
print("Step 2.5 Complete!")
print("="*60)

Step 2.5: Create RSICD Evaluation Config
✓ Created RSICD eval config: /content/drive/MyDrive/data/RSGPT/eval_configs/rsicd_eval.yaml

Step 2.5 Complete!


# STEP 5: FINE-TUNE ON RSICD

In [ ]:
# @title Check RSGPT Pretrained Weight Loading
import os

rsgpt_path = "/content/drive/MyDrive/data/RSGPT"

# Search for pretrained weight loading logic
print("Searching for pretrained weight loading in model code...")
!grep -r "pretrained\|instruct_blip\|load_pretrained" {rsgpt_path}/rsgpt/models --include="*.py" -A 3 | head -60

# Check model configs for default pretrained path
print("\n" + "="*60)
print("Checking model configs for default pretrained paths...")
!grep -r "pretrained" {rsgpt_path}/configs/models --include="*.yaml"

Searching for pretrained weight loading in model code...
/content/drive/MyDrive/data/RSGPT/rsgpt/models/Qformer.py:    An abstract class to handle weights initialization and a simple interface for downloading and loading pretrained
/content/drive/MyDrive/data/RSGPT/rsgpt/models/Qformer.py-    models.
/content/drive/MyDrive/data/RSGPT/rsgpt/models/Qformer.py-    """
/content/drive/MyDrive/data/RSGPT/rsgpt/models/Qformer.py-
--
/content/drive/MyDrive/data/RSGPT/rsgpt/models/Qformer.py:            >>> tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
/content/drive/MyDrive/data/RSGPT/rsgpt/models/Qformer.py:            >>> config = BertConfig.from_pretrained("bert-base-cased")
/content/drive/MyDrive/data/RSGPT/rsgpt/models/Qformer.py:            >>> model = BertLMHeadModel.from_pretrained('bert-base-cased', config=config)
/content/drive/MyDrive/data/RSGPT/rsgpt/models/Qformer.py-            >>> inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
/content/drive/MyDr

In [ ]:
# @title Step 2.6: Fine-tune on RSICD
import torch
import gc

print("="*60)
print("Step 2.6: Fine-tune on RSICD")
print("="*60)

# Clear GPU memory
print("\n[1] Clearing GPU memory...")
torch.cuda.empty_cache()
gc.collect()
print("  ✓ GPU memory cleared")

# Check GPU
print("\n[2] Checking GPU...")
!nvidia-smi

# Change to RSGPT directory
%cd /content/drive/MyDrive/data/RSGPT

# Start training
print("\n[3] Starting RSICD fine-tuning...")
print("  This may take 1-2 hours depending on your GPU")
print("="*60)

# !torchrun --nproc_per_node=1 train.py --cfg-path train_configs/rsicd_train.yaml
!torchrun --nproc_per_node=1 train.py --cfg-path train_configs/rsicd_direct_train.yaml


Step 2.6: Fine-tune on RSICD

[1] Clearing GPU memory...
  ✓ GPU memory cleared

[2] Checking GPU...
Tue Dec 30 17:19:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   35C    P0             54W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                      

# STEP 6: EVALUATE ON RSICD TEST SET

In [ ]:
%cd /content/drive/MyDrive/data/RSGPT

/content/drive/MyDrive/data/RSGPT


In [ ]:
# @title Step 2.7: Evaluate on RSICD Test Set (Final Version)
import json, os, glob, argparse
from PIL import Image
from pycocotools.coco import COCO
import sys

# Change to RSGPT directory FIRST
os.chdir("/content/drive/MyDrive/data/RSGPT")
sys.path.insert(0, "/content/drive/MyDrive/data/RSGPT")

import torch
from rsgpt.common.config import Config
from rsgpt.common.registry import registry
from rsgpt.datasets.builders import *
from rsgpt.models import *
from rsgpt.processors import *

from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

print("="*60)
print("RSICD Test Set Evaluation")
print("="*60)

# Install Java if needed
import subprocess
result = subprocess.run(['which', 'java'], capture_output=True, text=True)
if not result.stdout.strip():
    print("Installing Java for METEOR...")
    !apt-get update -qq
    !apt-get install -y -qq default-jdk
    print("✓ Java installed")

STOP_TOKEN = "###"

def clean_caption(caption):
    if STOP_TOKEN in caption:
        caption = caption.split(STOP_TOKEN)[0]
    return caption.strip()

# ============================================
# Paths - UPDATE THESE AS NEEDED
# ============================================
RSICD_PATH = "/content/drive/MyDrive/data/RSGPT/dataset/rsicd"
# CHECKPOINT_DIR = "/content/drive/MyDrive/outputs/rsgpt/rsicd_direct"  # or rsicd_finetuned_v2
CHECKPOINT_DIR = "/content/drive/MyDrive/outputs/rsgpt/rsicd_finetuned"
EVAL_CONFIG = "eval_configs/rsicd_eval.yaml"

# ============================================
# [1] Load RSICD Test Data
# ============================================
print("\n[1] Loading RSICD test data...")

test_file = f"{RSICD_PATH}/rsicd_cap_processed_instruction_test.json"
with open(test_file) as f:
    test_data = json.load(f)["annotations"]
print(f"  ✓ Loaded {len(test_data)} test images")

# Calculate average GT length
total_words = sum(len(cap.split()) for item in test_data for cap in item["text_output"])
total_caps = sum(len(item["text_output"]) for item in test_data)
avg_gt_len = total_words / total_caps
print(f"  Average GT caption length: {avg_gt_len:.1f} words")

# ============================================
# [2] Load COCO GT
# ============================================
print("\n[2] Loading COCO GT...")
coco = COCO(f"{RSICD_PATH}/rsicd_test_gt.json")
print("  ✓ COCO GT loaded")

# ============================================
# [3] Find Checkpoints
# ============================================
print("\n[3] Finding checkpoints...")
ckpts = sorted(glob.glob(f"{CHECKPOINT_DIR}/*/checkpoint_*.pth"))
print(f"  ✓ Found {len(ckpts)} checkpoints:")
for i, c in enumerate(ckpts):
    print(f"    [{i+1}] {os.path.basename(c)}")

if not ckpts:
    print("  ❌ No checkpoints found! Check CHECKPOINT_DIR path.")

# ============================================
# [4] Load Model
# ============================================
print("\n[4] Loading model...")
%cd /content/drive/MyDrive/data/RSGPT

cfg = Config(argparse.Namespace(cfg_path=EVAL_CONFIG, options=None))
model_cls = registry.get_model_class(cfg.model_cfg.arch)
model = model_cls.from_config(cfg.model_cfg).cuda()

# Setup processor
print("  Setting up visual processor...")
try:
    vis_processor_cfg = cfg.datasets_cfg.rsicd_instruction.vis_processor.train
    vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
    print(f"  ✓ Using processor: {vis_processor_cfg.name}")
except:
    try:
        vis_processor_cfg = cfg.datasets_cfg.rsicap_instruction.vis_processor.train
        vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
        print(f"  ✓ Using processor: {vis_processor_cfg.name}")
    except:
        from torchvision import transforms
        vis_processor = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                               std=[0.26862954, 0.26130258, 0.27577711])
        ])
        print("  ✓ Using fallback processor")

print("  ✓ Model loaded")

# ============================================
# [5] Evaluate Each Checkpoint
# ============================================
# Short prompt for short captions
EVAL_PROMPT = "Briefly describe the content of the image."

best = {"ckpt": None, "CIDEr": -1.0, "metrics": None}
all_results = []

for ckpt_idx, ckpt in enumerate(ckpts):
    print(f"\n{'='*60}")
    print(f"[Checkpoint {ckpt_idx+1}/{len(ckpts)}] {os.path.basename(ckpt)}")
    print("="*60)

    # Load weights
    print("  Loading checkpoint weights...")
    checkpoint = torch.load(ckpt, map_location="cuda")
    model.load_state_dict(checkpoint["model"], strict=False)
    model.eval()
    epoch = checkpoint.get('epoch', '?')
    print(f"  ✓ Loaded epoch {epoch}")

    preds = []
    total_images = len(test_data)
    print(f"\n  Generating captions for {total_images} images...")

    with torch.no_grad():
        for img_idx, item in enumerate(test_data):
            if (img_idx + 1) % 200 == 0 or img_idx == 0:
                print(f"    Processing {img_idx+1}/{total_images} ({100*(img_idx+1)/total_images:.1f}%)")

            # Note: RSICD uses 'image/' folder (not 'images/')
            img_path = f"{RSICD_PATH}/image/{item['filename']}"
            image = vis_processor(Image.open(img_path).convert("RGB")).unsqueeze(0).cuda()

            prompt = item.get("text_input", EVAL_PROMPT)

            # ============================================
            # SHORT caption generation for RSICD
            # ============================================
            caption = model.generate(
                {"image": image, "prompt": prompt},
                use_nucleus_sampling=False,
                num_beams=5,
                max_length=80,           # SHORT - RSICD ~10-15 words
                min_length=8,
                repetition_penalty=1.2,
                length_penalty=1.0,
                num_captions=1,
            )[0]

            caption = clean_caption(caption)
            preds.append({"image_id": int(item["image_id"]), "caption": caption})

    print(f"  ✓ Generated {len(preds)} captions")

    # Show samples
    print("\n  Sample predictions:")
    for i in range(min(3, len(preds))):
        gt_caps = [ann["caption"] for ann in coco.imgToAnns.get(preds[i]["image_id"], [])]
        gen_words = len(preds[i]['caption'].split())
        gt_words = len(gt_caps[0].split()) if gt_caps else 0
        print(f"\n    Image {preds[i]['image_id']}:")
        print(f"      Generated ({gen_words} words): {preds[i]['caption']}")
        print(f"      GT ({gt_words} words): {gt_caps[0] if gt_caps else 'N/A'}")

    # Length stats
    avg_len = sum(len(p["caption"].split()) for p in preds) / len(preds)
    print(f"\n  Avg generated length: {avg_len:.1f} words (GT: {avg_gt_len:.1f})")

    # ============================================
    # [6] Compute Metrics
    # ============================================
    print("\n  Computing metrics...")

    gts = {}
    res = {}
    for pred in preds:
        img_id = pred["image_id"]
        gts[img_id] = [ann["caption"] for ann in coco.imgToAnns.get(img_id, [])]
        res[img_id] = [pred["caption"]]

    scorers = [
        (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
        (Meteor(), "METEOR"),
        (Rouge(), "ROUGE_L"),
        (Cider(), "CIDEr"),
    ]

    metrics = {}
    for scorer, method in scorers:
        score, _ = scorer.compute_score(gts, res)
        if isinstance(method, list):
            for m, s in zip(method, score):
                metrics[m] = s
        else:
            metrics[method] = score

    cider = metrics.get("CIDEr", 0.0)

    print(f"\n  Results:")
    print(f"    BLEU-1:  {metrics.get('Bleu_1', 0):.4f}")
    print(f"    BLEU-2:  {metrics.get('Bleu_2', 0):.4f}")
    print(f"    BLEU-3:  {metrics.get('Bleu_3', 0):.4f}")
    print(f"    BLEU-4:  {metrics.get('Bleu_4', 0):.4f}")
    print(f"    METEOR:  {metrics.get('METEOR', 0):.4f}")
    print(f"    ROUGE-L: {metrics.get('ROUGE_L', 0):.4f}")
    print(f"    CIDEr:   {cider:.4f}")

    all_results.append({
        "checkpoint": ckpt,
        "epoch": epoch,
        "avg_caption_length": avg_len,
        "metrics": metrics
    })

    if cider > best["CIDEr"]:
        best = {"ckpt": ckpt, "CIDEr": cider, "metrics": metrics, "epoch": epoch}
        print(f"\n  >>> ★ NEW BEST CHECKPOINT!")

# ============================================
# [7] Save Results
# ============================================
results_path = f"{CHECKPOINT_DIR}/evaluation_results_with_METEOR.json"
with open(results_path, "w") as f:
    json.dump({"best": best, "all_results": all_results}, f, indent=2)

# ============================================
# [8] Final Summary
# ============================================
print("\n" + "="*60)
print("EVALUATION COMPLETE")
print("="*60)
print(f"\nBest checkpoint: {os.path.basename(best['ckpt'])}")
print(f"Best epoch: {best.get('epoch', '?')}")
print(f"\nBest Results:")
if best["metrics"]:
    for k, v in best["metrics"].items():
        print(f"  {k}: {v:.4f}")

# Compare with paper
print("\n" + "-"*60)
print("Comparison with RSGPT Paper (Table V):")
print("-"*60)
paper = {"Bleu_1": 0.7032, "Bleu_2": 0.5422, "Bleu_3": 0.4353, "Bleu_4": 0.3683,
         "METEOR": 0.3010, "ROUGE_L": 0.5334, "CIDEr": 1.0294}

print(f"{'Metric':<10} {'Yours':>10} {'Paper':>10} {'Diff':>10}")
print("-"*42)
for m in ["Bleu_1", "Bleu_4", "METEOR", "ROUGE_L", "CIDEr"]:
    yours = best["metrics"].get(m, 0)
    theirs = paper[m]
    diff = yours - theirs
    print(f"{m:<10} {yours:>10.4f} {theirs:>10.4f} {diff:>+10.4f}")

print(f"\nResults saved to: {results_path}")

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

# STEP 7: COMPARE WITH PAPER RESULTS

In [ ]:
# @title Step 2.8: Compare with Paper Results
import json

print("="*60)
print("Step 2.8: Compare with Paper Results")
print("="*60)

# Load your results
results_path = "/content/drive/MyDrive/outputs/rsgpt/rsicd_finetuned/evaluation_results.json"
with open(results_path) as f:
    results = json.load(f)

your_metrics = results["best"]["metrics"]

# Paper results (from RSGPT paper Table V)
paper_metrics = {
    "Bleu_1": 0.7032,
    "Bleu_2": 0.5422,
    "Bleu_3": 0.4353,
    "Bleu_4": 0.3683,
    "METEOR": 0.3010,
    "ROUGE_L": 0.5334,
    "CIDEr": 1.0294
}

print("\n" + "="*60)
print("RESULTS COMPARISON")
print("="*60)
print(f"\n{'Metric':<12} {'Your Result':>12} {'Paper Result':>14} {'Difference':>12}")
print("-"*52)

for metric in ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4", "METEOR", "ROUGE_L", "CIDEr"]:
    yours = your_metrics.get(metric, 0)
    paper = paper_metrics.get(metric, 0)
    diff = yours - paper
    diff_str = f"{diff:+.4f}" if diff != 0 else "0.0000"

    # Status indicator
    if yours >= paper * 0.95:
        status = "✅"
    elif yours >= paper * 0.85:
        status = "⚠️"
    else:
        status = "❌"

    print(f"{metric:<12} {yours:>12.4f} {paper:>14.4f} {diff_str:>12} {status}")

print("-"*52)

# Overall assessment
avg_ratio = sum(your_metrics.get(m, 0) / paper_metrics[m] for m in paper_metrics) / len(paper_metrics)
print(f"\nOverall performance: {avg_ratio*100:.1f}% of paper results")

if avg_ratio >= 0.95:
    print("🎉 Excellent! Your results match the paper!")
elif avg_ratio >= 0.85:
    print("👍 Good! Your results are close to the paper.")
elif avg_ratio >= 0.70:
    print("⚠️ Your results are below expectations. Consider:")
    print("   - Training longer")
    print("   - Adjusting hyperparameters")
    print("   - Checking data preprocessing")
else:
    print("❌ Results need improvement. Check training setup.")

Step 2.8: Compare with Paper Results

RESULTS COMPARISON

Metric        Your Result   Paper Result   Difference
----------------------------------------------------
Bleu_1             0.6759         0.7032      -0.0273 ✅
Bleu_2             0.4948         0.5422      -0.0474 ⚠️
Bleu_3             0.3792         0.4353      -0.0561 ⚠️
Bleu_4             0.3040         0.3683      -0.0643 ❌
METEOR             0.0000         0.3010      -0.3010 ❌
ROUGE_L            0.5296         0.5334      -0.0038 ✅
CIDEr              0.8917         1.0294      -0.1377 ⚠️
----------------------------------------------------

Overall performance: 77.6% of paper results
⚠️ Your results are below expectations. Consider:
   - Training longer
   - Adjusting hyperparameters
   - Checking data preprocessing
